In [287]:
import pandas as pd
import numpy as np
df_incidents = pd.concat([pd.read_excel('./incidents.xlsx', sheet_name='Result 1'), pd.read_excel('./incidents.xlsx', sheet_name='Result 2')],ignore_index=True)

In [288]:
df = df_incidents
df = df.rename(columns={
    "Наименование":"name", 
    "Дата создания во внешней системе":"date_system_start", 
    "Дата закрытия":"date_end", 
    "Дата и время завершения события во":"date_system_end", 
    "Округ":"area", 
    "Источник": "source",
    "Адрес":"address", 
    })
df[['date_system_start', 'date_system_end', 'date_end']] =df[['date_system_start', 'date_system_end', 'date_end']].apply(pd.to_datetime) 


In [289]:
df['date_system_end'] = np.where(df['date_system_end'].isna(), df['date_end'], df['date_system_end'])
df = df.drop(['area', 'date_end', 'address'], axis=1)

In [290]:
def set_source_priority(name_source: str):
    match name_source:
        case 'ASUPR': return 0.1
        case 'CAFAP': return 0.3
        case 'EDC': return 0.3
        case 'EVAGD': return 0.2
        case 'GORMOST': return 0.2
        case 'KGH': return 0.6
        case 'MGI': return 0.5
        case 'MOS_GAS': return 1.0
        case 'MVK': return 0.1
        case 'NG': return 0.3
df['source_priority'] = df['source'].apply(lambda x: set_source_priority(x))

In [291]:
df['delta_time_in_seconds'] = df['date_system_end'] - df['date_system_start']
df['delta_time_in_seconds']=df['delta_time_in_seconds'].astype('int64') // 10**9
df_only_instance = df[['name', 'source', 'delta_time_in_seconds', 'source_priority']]
df_only_instance.loc[df_only_instance['delta_time_in_seconds'] <= 0, ['delta_time_in_seconds']] = 1
df_only_instance.loc[:,['incident_prority']] =  df_only_instance['delta_time_in_seconds'] / df_only_instance['delta_time_in_seconds'].max()
df_only_instance = df_only_instance.drop(['delta_time_in_seconds'], axis=1)
df_only_instance['priority'] = df_only_instance['incident_prority'] * df_only_instance['source_priority']
df_only_instance.sort_values(by=['priority'])

df_only_instance = df_only_instance.drop(['incident_prority', 'source_priority'], axis=1)
df_only_instance.sort_values(by=['priority'])


,name,source,priority
496253,Отсутствуют актуальные мгновенные значения,ASUPR,0.000000
457510,Нет связи с ПУ,ASUPR,0.000000
457503,Нет связи с ПУ,ASUPR,0.000000
457500,Нет связи с ПУ,ASUPR,0.000000
457483,Нет связи с ПУ,ASUPR,0.000000
...,...,...,...
649490,Запах газа в кухне,MOS_GAS,0.655190
404094,Запах газа в кухне,MOS_GAS,0.659276
32327,Запах газа в кухне,MOS_GAS,0.662731
461956,Запах газа в кухне,MOS_GAS,0.676802


In [308]:
df_result = df_only_instance.groupby(by=['name', 'source'], as_index=False).mean()
df_result = df_result.sort_values(by=['priority'], ascending=[False]).reset_index(drop=True)
df_result

,name,source,priority
0,Запах газа в квартире (помещении); Запах газа ...,MOS_GAS,0.605363
1,Занижен колодец,MOS_GAS,0.570183
2,"Запах газа на улице, в подъезде",MOS_GAS,0.547578
3,Запах газа от стояка,MOS_GAS,0.443802
4,Запах газа на улице и в подъезде,MOS_GAS,0.436469
...,...,...,...
357,Гул (шум) на объекте АО МОСГАЗ; Запах газа на ...,MOS_GAS,0.000005
358,Нарушение в работе газового оборудования; Слаб...,MOS_GAS,0.000005
359,Угроза взрыва; Запах газа в доме,MOS_GAS,0.000002
360,Прорыв теплосети (водопровода); Открыт колодец,MOS_GAS,0.000000


In [309]:
words_coef = {
    5 : ['пожар','хлопок','взрыв', 'затоплен', 'дым', 'газ'],
}
for coef in words_coef.keys():
    for word in words_coef[coef]:
        # print(f"{coef} - {word}")
        df_result.loc[df_result['name'].str.contains(word),['priority']] *= coef 

df_result.loc[:,['priority']] = df_result['priority'] / df_result['priority'].max()
df_result

,name,source,priority
0,Запах газа в квартире (помещении); Запах газа ...,MOS_GAS,1.000000
1,Занижен колодец,MOS_GAS,0.188377
2,"Запах газа на улице, в подъезде",MOS_GAS,0.904545
3,Запах газа от стояка,MOS_GAS,0.733117
4,Запах газа на улице и в подъезде,MOS_GAS,0.721004
...,...,...,...
357,Гул (шум) на объекте АО МОСГАЗ; Запах газа на ...,MOS_GAS,0.000008
358,Нарушение в работе газового оборудования; Слаб...,MOS_GAS,0.000008
359,Угроза взрыва; Запах газа в доме,MOS_GAS,0.000020
360,Прорыв теплосети (водопровода); Открыт колодец,MOS_GAS,0.000000


In [311]:
df_result.sort_values(by=['priority'], ascending=[False]).to_excel('incident_priority.xlsx', index=False)